In [ ]:
import numpy as np
import torch
from torch import nn
import pandas as pd
from tqdm import tqdm
from transformers import logging
logging.set_verbosity_error()
from transformers import BertModel, BertTokenizer, BertForMaskedLM, pipeline
from sentence_transformers import SentenceTransformer, util

print(torch.cuda.is_available())
print(torch.__version__)

import os
from pathlib import Path
# os.chdir(Path(os.environ["ALTHOME"]) / "Master-thesis")

In [ ]:
l = ["Ä","ä","Ö","ö","Ü","ü","ß"]
replacements = {key: str(bytes(key, encoding="utf-8"), encoding="latin-1") for key in l}

def replace(s):
    for k,v in replacements.items():
        s = s.replace(v, k)
    return s

if False:
    with open("data/SimpleGerman/spacy_easy.txt") as fp:
        text = fp.read()
    text = replace(text)
    with open("data/SimpleGerman/fixed_easy.txt", "w") as fp:
        fp.write(text)
    with open("data/SimpleGerman/spacy_normal.txt") as fp:
        text = fp.read()
    text = replace(text)
    with open("data/SimpleGerman/fixed_normal.txt", "w") as fp:
        fp.write(text)

In [ ]:
model_name = "dbmdz/bert-base-german-cased"
model_name = "deepset/gbert-base"
sentence = "Dies ist ein Satz"
model_name = "deepset/gbert-base"

pipe  = pipeline(model=model_name, tokenizer=model_name, task="feature-extraction")
model_encode = BertModel.from_pretrained(model_name)
model_decode = BertModel.from_pretrained(model_name, is_decoder=True, add_cross_attention=True)
model_LM = BertForMaskedLM.from_pretrained(model_name)
tokenizer = BertTokenizer.from_pretrained(model_name)


In [ ]:
tokens = tokenizer.encode(sentence, return_tensors="pt")
output = model_encode.forward(tokens, output_hidden_states=True)
output_pipe = pipe(sentence)

a = output.hidden_states[-1].detach().numpy() # same as pipeline feature extraction
c = output.last_hidden_state.detach().numpy()
b = np.array(output_pipe)

print((a == b).all())
print((a == c).all())
print(f"{type(a)} - {a.shape}")
print(f"{type(b)} - {b.shape}")
print(f"{type(c)} - {c.shape}")

In [ ]:
tokenizer = BertTokenizer.from_pretrained(model_name)
tokens = tokenizer.encode(sentence, return_tensors="pt")
output_encode = model_encode.forward(tokens)
output_LM = model_LM(tokens, output_hidden_states=True)


print(output_encode.last_hidden_state.size())
print(output_encode.pooler_output.size())
print(output_encode.keys())
print(output_LM.keys())
print((output_encode.last_hidden_state == output_LM.hidden_states[-1]))

In [ ]:
s = "Dies ist ein Test"
model_name="deepset/gbert-base"
tokenizer = BertTokenizer.from_pretrained(model_name)
encoding = tokenizer.encode(s, padding="max_length")
decoding = tokenizer.decode(encoding)

print(len(encoding))
print(encoding)
print(decoding)

In [ ]:
d_model = 512
tgt_mask = torch.triu(torch.ones(9, 9) * float('-inf'), diagonal=1)

encoder_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=8,batch_first=True)
transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=6)
decoder_layer = nn.TransformerDecoderLayer(d_model=d_model, nhead=8,batch_first=True)
transformer_decoder = nn.TransformerDecoder(decoder_layer, num_layers=6)

src = torch.rand(32, 10, d_model)
memory = transformer_encoder(src)
output = transformer_decoder(src[:,:-1,:],memory,tgt_mask=tgt_mask)
print(src.shape)
print(memory.shape)
print(output.shape)